<a href="https://colab.research.google.com/github/jenyadev3/pandas_tasks/blob/main/task_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Будем работать с базой данных из Национального института диабета. Целью набора данных явялется диагностическое прогнозирование диабета у пациентов.Несколько ограничений были наложены, в частности, все пациенты здесь - женщины в возрасте от 21 года индийского происхождения.

1. Pregnancies - отображает кол-во беременностей. Представлено целым числом
2. Glucose - уровень глюкозы, целое число
3. BloodPressure - давление, миллиметры р/с, целвые числа
4. SkinThickness - обхват трицепса в мм, целое число
5. Insulin - уровень инсулина, целое число
6. BMI - индекс массы тела
7. DiabetesPedigreeFunction - риск наследственного диабета, десятичная дробь от 0 до 1
8. Age - возраст в целых числах
9. Class - наличие диабета, где 0 - здоров, 1 - болен







In [6]:
import pandas as pd

df = pd.read_csv("/content/diabet.csv")
df.head(1)

# переименование столбцов
df.columns = ["pregnancies", "glucose", "blood_pressure", "skin_thickness","insulin","bmi","diabet_function","age", "disease"]
df.head(1)

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabet_function,age,disease
0,6,148.0,72.0,35.0,125.0,33.6,0.627,50,1


In [10]:
# вычисляем основные статистики для всех столбцов
df.agg(["max","min","mean","var"])
# Для некоторых
df[["pregnancies","glucose"]].agg(["max","min","mean","var"])

,pregnancies,glucose
max,17.000000,199.000000
min,0.000000,44.000000
mean,3.845052,121.656250
var,11.354056,926.489244


In [18]:
# найдем у кого старше 50 лет обнаружен диабет:
# способ 1
flt = (df.age > 50) & (df.disease == 1)
df.loc[flt].shape[0]
print(f"У {df.loc[flt].shape[0]} женщин")
# способ 2
df.query("age > 50 & disease == 1").shape[0]

У 38 женщин


38

In [28]:
# найти трех женщин с наибольшим количеством беременностей
# способ 1
df.sort_values(by="pregnancies",ascending=False).head(3)
# способ 2
top_3 = df.pregnancies.nlargest(3).to_list()
flt = df.pregnancies.isin(top_3)
df.loc[flt]

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabet_function,age,disease
88,15,136.0,70.0,32.0,110.0,37.1,0.153,43,1
159,17,163.0,72.0,41.0,114.0,40.9,0.817,47,1
298,14,100.0,78.0,25.0,184.0,36.6,0.412,46,1
455,14,175.0,62.0,30.0,125.0,33.6,0.212,38,1


In [35]:
# нормальным давлением считается 80-89. у скольких % оно нормальное (Женщины с нормальным давлением / все женщины * 100)
flt = (df.blood_pressure.between (80,89))
float(round(df.loc[flt].shape[0]/df.shape[0]*100,2))

18.88

In [33]:
# сколько женщин в возрасте от 30 до 40 лет успело родить 3 и более детей
flt = (df.pregnancies >= 3) & (df.age.between (30,40))
df.loc[flt].shape[0]

142

In [39]:
# добавить новый бинарный признак: была ли женщина беременна или нет
import numpy as np

df["wasPregnant"] = np.where(df.pregnancies>0,1,0)

df[["pregnancies", "wasPregnant"]].head(10)

,pregnancies,wasPregnant
0,6,1
1,1,1
2,8,1
3,1,1
4,0,0
5,5,1
6,3,1
7,10,1
8,2,1
9,8,1


In [42]:
# новый столбец тип тела
df["BodyType"] = np.where(df.bmi <= 18.5, "Underweight",
np.where(((df.bmi > 18.5) & (df.bmi <= 24.9)),"Normal weight",
np.where(((df.bmi > 25) & (df.bmi <= 29.9)), "Overweight", "Obesity")))
df[["bmi", "BodyType"]].head(10)

,bmi,BodyType
0,33.600000,Obesity
1,26.600000,Overweight
2,23.300000,Normal weight
3,28.100000,Overweight
4,43.100000,Obesity
5,25.600000,Overweight
6,31.000000,Obesity
7,35.300000,Obesity
8,30.500000,Obesity
9,32.457464,Obesity


In [48]:
# для каждого типа тела посчитать mean беременность, min и max глюкоза
df.groupby("BodyType").agg(min_preg = ("pregnancies","mean"),min_gluc = ("glucose","min"),max_gluc = ("glucose","max"))

,min_preg,min_gluc,max_gluc
BodyType,,,
Normal weight,3.245098,56.0,194.0
Obesity,4.014315,44.0,199.0
Overweight,3.791908,71.0,197.0
Underweight,0.750000,83.0,104.0


In [57]:
df_1 = (df.groupby(["BodyType","disease"]).agg(min_preg = ("pregnancies","mean"),min_gluc = ("glucose","min"),max_gluc = ("glucose","max")))
df_1
# для больных и здоровых женщин каждого типа рассчитать mean blood_pressure
df.pivot_table(index = "BodyType", columns = "disease", values = "blood_pressure", aggfunc = "mean")

disease,0,1
BodyType,,
Normal weight,67.894737,71.142857
Obesity,73.078358,75.760181
Overweight,68.774436,72.300000
Underweight,69.500000,NaN
